### Getting the Dataset

http://www.bbci.de/competition/iii/desc_IVa.html

In [1]:
from __future__ import division

import numpy as np
import scipy as sp
from scipy.io import loadmat
import matplotlib as mpl
from wyrm import processing as proc
from wyrm.types import Data
from wyrm import plot
from matplotlib import pyplot as plt

from os import path

DEBUG:matplotlib:CACHEDIR=/home/nahuel/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/nahuel/.cache/matplotlib/fontlist-v310.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
#Se carga data txt Class-Time x Channel (298458 x 118)
#    #tr  #te
#aa  168 112
#al  224  56
#av  84  196
#aw  56  224
#ay  28  252

# /home/nahuel/Documents/TESIS/DATA/Competencia BCI/III/Dataset_IV/txt

sujeto='aa'
nro_tr=168
data_cnt_txt = np.loadtxt('../../DATA/Competencia BCI/III/Dataset_IV/txt/'+sujeto+'/100Hz/data_set_IVa_'+sujeto+'_cnt.txt')
#Traspuesta Channel x Class-Time (118 x 298458)
data_cnt_txt = data_cnt_txt.transpose()

#Se carga 280 etiquetas de Class
data_lab_txt = np.loadtxt('../../DATA/Competencia BCI/III/Dataset_IV/txt/'+sujeto+'/100Hz/data_set_IVa_'+sujeto+'_mrk.txt')

In [3]:
#Se carga 100 etiquetas TRUE LABELS
TRUE_LABELS = '../../DATA/Competencia BCI/III/Dataset_IV/txt/'+sujeto+'/TRUE_LABELS.txt'

true_labels = np.loadtxt(TRUE_LABELS).astype('int')
#Convertimos los -1 a 0
true_labels[true_labels == 2] = 0

true_labels = true_labels[nro_tr:]


In [4]:
#Se castea a Int (1- Left, 2- Foot, 0- Test)
data_lab_txt = np.nan_to_num(data_lab_txt)
data_lab_txt = data_lab_txt.astype('int')

#La primer columna es de posiciones de eventos
posicion = data_lab_txt[:,0]
#La segunda columna es de labels
data_lab = data_lab_txt[:,1]

flag=1
for pos in posicion:
    #Nos quedamos con muestras de 60 frames por evento para los 118 channels
    aux = np.array( [ data_cnt_txt[:, pos-30:pos+30] ] )
    if flag == 1:
        data_cnt=aux
        flag=0
    else:
        data_cnt=np.append(data_cnt, aux, axis=0)
        

#Modificamos el orden de las dimensiones como Class x Time x Channel (280 x 60 x 118)
data_cnt=data_cnt.swapaxes(-1, -2)

#Los primeros 168 Class son de train como Class x Time x Channel (168 x 60 x 118)
data_train=data_cnt[:nro_tr]
#El resto de las Class son de test como Class x Time x Channel (112 x 60 x 118)
data_test=data_cnt[nro_tr:]


#Las labes validas son para el conjunto de train
data_lab = data_lab[:nro_tr]
data_lab[data_lab == 2] = 0


In [5]:
#Convertimos a wyrm Data - Set Train
#Cargamos data training
data = data_train
labels = data_lab

#Creamos axes con enumerados de la misma longitud que data Class x Time x Channel
axes = [np.arange(i) for i in data.shape]
#Se completa la dimension Class con labels
axes[0] = labels
#Se convierte a str la dimension Channel
#axes[2] = [str(i) for i in range(data.shape[2])]
axes[2] = ['Fp1', 'AFp1', 'Fpz', 'AFp2', 'Fp2', 'AF7', 'AF3', 'AF4', 'AF8', 'FAF5', 'FAF1', 'FAF2', 'FAF6', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FFC7', 'FFC5', 'FFC3', 'FFC1', 'FFC2', 'FFC4', 'FFC6', 'FFC8', 'FT9', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'FT10', 'CFC7', 'CFC5', 'CFC3', 'CFC1', 'CFC2', 'CFC4', 'CFC6', 'CFC8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'CCP7', 'CCP5', 'CCP3', 'CCP1', 'CCP2', 'CCP4', 'CCP6', 'CCP8', 'TP9', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'TP10', 'PCP7', 'PCP5', 'PCP3', 'PCP1', 'PCP2', 'PCP4', 'PCP6', 'PCP8', 'P9', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'P10', 'PPO7', 'PPO5', 'PPO1', 'PPO2', 'PPO6', 'PPO8', 'PO7', 'PO3', 'PO1', 'POz', 'PO2', 'PO4', 'PO8', 'OPO1', 'OPO2', 'O1', 'Oz', 'O2', 'OI1', 'OI2', 'I1', 'I2' ]

names = ['Class', 'Time', 'Channel']
units = ['#', 'ms', '#']
dat_train = Data(data=data, axes=axes, names=names, units=units)
dat_train.fs = 100
dat_train.class_names = ['Left', 'Foot']


In [6]:
#Convertimos a wyrm Data - Set Test
#Cargamos data test
data = data_test

#Creamos axes con enumerados de la misma longitud que data Class x Time x Channel
axes = [np.arange(i) for i in data.shape]
#Se convierte a str la dimension Channel
axes[2] = [str(i) for i in range(data.shape[2])]
names = ['Epoch', 'Time', 'Channel']
units = ['#', 'ms', '#']
dat_test = Data(data=data, axes=axes, names=names, units=units)
dat_test.fs = 100


In [7]:
def plot_csp_pattern(a):
    # get symmetric min/max values for the color bar from first and last column of the pattern
    maxv = np.max(np.abs(a[:, [0, -1]]))
    minv = -maxv
    
    im_args = {'interpolation' : 'None', 
           'vmin' : minv, 
           'vmax' : maxv
           }

    # plot
    ax1 = plt.subplot2grid((1,11), (0,0), colspan=5)
    ax2 = plt.subplot2grid((1,11), (0,5), colspan=5)
    ax3 = plt.subplot2grid((1,11), (0,10))

    ax1.imshow(a[:, 0].reshape(8, 8), **im_args)
    ax1.set_title('Pinky')

    ax = ax2.imshow(a[:, -1].reshape(8, 8), **im_args)
    ax2.set_title('Tongue')

    plt.colorbar(ax, cax=ax3)
    plt.tight_layout()

In [8]:
def filtros(data):
    dat = data.copy()
    
    #Mitad de la frecuencia
    fs_n = dat.fs / 2
    
    #Se aplica filtros
    b, a = proc.signal.butter(5, [13 / fs_n], btype='low')   
    dat = proc.filtfilt(dat, b, a)
    
    b, a = proc.signal.butter(5, [9 / fs_n], btype='high')
    dat = proc.filtfilt(dat, b, a)
    return dat
    

In [9]:
def preprocess(data, filt=None):
    #Se aplican los filtros
    dat=filtros(data)
    
    #Se toma una muestra de 50 cada 1000 (fs_n = 1000) (278 x 150 x 64)
    dat = proc.subsample(dat, 50)
        
    if filt is None:
        #Calcula el patrón espacial común (CSP) para dos clases.
        filt, pattern, _ = proc.calculate_csp(dat)
        #Grafica csp_pattern
        #plot_csp_pattern(pattern)
        
    #Aplica el CSP
    dat = proc.apply_csp(dat, filt)
    #print(filt[0])
    dat = proc.variance(dat)
    dat = proc.logarithm(dat)
    return dat, filt

In [10]:
#Se calcula el vector de caracteristicas para los conjuntos de train y test
fv_train, filt = preprocess(dat_train)
fv_test, _ = preprocess(dat_test, filt)

In [11]:
#dat_new = proc.remove_channels(dat_train, ['PPO4','CP4'])
#proc.select_channels(dat_train, ['PPO4','CP4'])

In [12]:
#Entrenamiento con LDA clasificador.
cfy = proc.lda_train(fv_train)

#Aplicar vector de características al clasificador LDA.
result = proc.lda_apply(fv_test, cfy)

#Normaliza resultados
#Si es > 0 = 1 sino -1
result = (np.sign(result)).astype('int')
#Convertimos los -1 a 0
result[result == -1] = 0

#Se compara los resultados del modelo contra los TRUE LABELS
print ('LDA Accuracy %.2f%%' % ((result == true_labels).sum() / len(result)) ) 


LDA Accuracy 0.55%


In [13]:
plt.show()

In [14]:
from sklearn import metrics as met

print(true_labels.shape)
print(result.shape)

print( met.confusion_matrix(true_labels, result) )
print( "Recall: ", met.recall_score(true_labels, result, average=None) )
print( "Precision: ", met.precision_score(true_labels, result, average=None) )
print( "F1: ", met.f1_score(true_labels, result, average=None) )
print( "Acurracy: ", met.accuracy_score(true_labels, result) )

tn, fp, fn, tp = met.confusion_matrix(true_labels, result).ravel()
print(tn, fp, fn, tp)

print(met.classification_report(true_labels, result))



(112,)
(112,)
[[27 25]
 [25 35]]
Recall:  [0.51923077 0.58333333]
Precision:  [0.51923077 0.58333333]
F1:  [0.51923077 0.58333333]
Acurracy:  0.5535714285714286
27 25 25 35
              precision    recall  f1-score   support

           0       0.52      0.52      0.52        52
           1       0.58      0.58      0.58        60

    accuracy                           0.55       112
   macro avg       0.55      0.55      0.55       112
weighted avg       0.55      0.55      0.55       112



In [15]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
X_actual = [1, 1, 0, 1, 0, 0, 1, 0, 0, 0]
Y_predic = [1, 0, 1, 1, 1, 0, 1, 1, 0, 0]
results = confusion_matrix(X_actual, Y_predic)
print ('Confusion Matrix :')
print(results)
print ('Accuracy Score is',accuracy_score(X_actual, Y_predic))
print ('Classification Report : ')
print (classification_report(X_actual, Y_predic))
print('AUC-ROC:',roc_auc_score(X_actual, Y_predic))
print('LOGLOSS Value is',log_loss(X_actual, Y_predic))

Confusion Matrix :
[[3 3]
 [1 3]]
Accuracy Score is 0.6
Classification Report : 
              precision    recall  f1-score   support

           0       0.75      0.50      0.60         6
           1       0.50      0.75      0.60         4

    accuracy                           0.60        10
   macro avg       0.62      0.62      0.60        10
weighted avg       0.65      0.60      0.60        10

AUC-ROC: 0.625
LOGLOSS Value is 13.815750437193334
